In [1]:
import pandas as pd
data=pd.read_csv('Products.csv')

In [2]:
filtered_data = data[data['main_category'] == 'appliances']
filtered_data['sub_category'].unique()

array(['Air Conditioners', 'All Appliances',
       'Heating & Cooling Appliances', 'Kitchen & Home Appliances',
       'Refrigerators', 'Washing Machines'], dtype=object)

In [8]:
filtered_data.shape

(29954, 10)

In [4]:
filtered_data['tags'] = filtered_data['main_category'] + ' ' + filtered_data['sub_category'] + ' ' + filtered_data['ratings'].astype(str) + ' ' + filtered_data['discount_price'] + ' ' + filtered_data['actual_price']

# Optionally, you can drop the original columns if needed
df = filtered_data.drop(columns=['main_category', 'sub_category', 'ratings', 'discount_price', 'actual_price','image','link','no_of_ratings'])


C:\Users\SYED AZIZ AHMED\AppData\Local\Temp\ipykernel_16628\3245301995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['tags'] = filtered_data['main_category'] + ' ' + filtered_data['sub_category'] + ' ' + filtered_data['ratings'].astype(str) + ' ' + filtered_data['discount_price'] + ' ' + filtered_data['actual_price']


In [5]:
unique_data = df.drop_duplicates(subset='name')

# Print the first few rows of the unique data to verify
print(unique_data.head())

# If you want to reset the index after dropping duplicates
unique_data.reset_index(drop=True, inplace=True)

# Print the unique data's shape to see the count of unique items
print("Count of unique items:", unique_data.shape[0])

                                                name  \
0  Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...   
1  LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...   
2  LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...   
3  LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...   
4  Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...   

                                              tags  
0  appliances Air Conditioners 4.2 ₹32,999 ₹58,990  
1  appliances Air Conditioners 4.2 ₹46,490 ₹75,990  
2  appliances Air Conditioners 4.2 ₹34,490 ₹61,990  
3    appliances Air Conditioners 4 ₹37,990 ₹68,990  
4  appliances Air Conditioners 4.1 ₹34,490 ₹67,790  
Count of unique items: 19107


In [6]:
data=unique_data

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np

# Sample dataset (you should replace this with your actual data)


# Convert 'tags' to lowercase
data['tags'] = data['tags'].str.lower()

# Tokenization and stopword removal
stop_words = set(stopwords.words('english'))

def preprocess_tags(tags):
    words = word_tokenize(tags)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

data['tags'] = data['tags'].apply(preprocess_tags)

# Split your data into subsets (chunks)
chunk_size = 19107  # Define the size of each subset
chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

# Initialize a TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000)

# Create an empty matrix to store combined cosine similarities
combined_similarity_matrix = None

# Loop through the subsets and calculate vectorization and cosine similarities
for chunk in chunks:
    # Vectorization
    tfidf_matrix = tfidf.fit_transform(chunk['tags'])

    # Calculate cosine similarity within this chunk
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the dimensions of the current similarity matrix
    rows, cols = cosine_sim.shape

    # Combine the similarity matrices
    if combined_similarity_matrix is None:
        combined_similarity_matrix = cosine_sim
    else:
        # If the dimensions don't match, pad the smaller matrix with zeros
        max_rows = max(combined_similarity_matrix.shape[0], rows)
        max_cols = max(combined_similarity_matrix.shape[1], cols)
        
        combined_similarity_matrix = np.pad(
            combined_similarity_matrix, ((0, max_rows - combined_similarity_matrix.shape[0]), 
                                        (0, max_cols - combined_similarity_matrix.shape[1])), 'constant')
        cosine_sim = np.pad(cosine_sim, ((0, max_rows - rows), (0, max_cols - cols)), 'constant')
        
        combined_similarity_matrix += cosine_sim

# If you want to average the similarities, divide by the number of subsets
combined_similarity_matrix /= len(chunks)
print("Cosine Similarity Matrix Shape:", combined_similarity_matrix.shape)
# Now, combined_similarity_matrix contains the combined cosine similarities
print(combined_similarity_matrix)


C:\Users\SYED AZIZ AHMED\AppData\Local\Temp\ipykernel_16628\990830790.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].str.lower()
C:\Users\SYED AZIZ AHMED\AppData\Local\Temp\ipykernel_16628\990830790.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(preprocess_tags)


MemoryError: Unable to allocate 1.36 GiB for an array with shape (365077449,) and data type int32